Let's do all preprocessing required for fitting the seq-2-seq model here. We need to extract features for the subwindows. We also need to write generic code incase we need to do changes later depending on the performance of the seq-2-seq model.

For the seq-2-seq model, we do not have to touch the densities, and as the first step we should extract the features from each subwindow using the fine-tuned model. We still might need the densities later for the latent AR proces part, we will get there in stage 2. We will need to arange the subwindows in a stack though, so that the features could be extracted. We ahve done this sort of work in AR models, we will look into that too.

In [1]:
import numpy as np
import tensorflow as tf
import os
from skimage.transform import resize
import matplotlib.pyplot as plt

2025-06-03 17:43:18.427889: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-03 17:43:19.976304: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-03 17:43:19.976365: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-03 17:43:20.236663: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-03 17:43:20.679512: I tensorflow/core/platform/cpu_feature_guar

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

##### Train and Validation data

Let's first focus on train and validation data. Meaning blocks 101, 102, 203, 301, and 204. We will preprocess data required for seq-2-seq model, we do not have to worry currently about the stage 2 (densities). But how are we going to evaluate the seq-2-seq model? should we do that on the rest of the blocks? Therefore we might have to preprocess the test data too. Let's get block 0101 as the example, develop all functions for this, and work on preprocessing data

For train data, we need to prepare our final input data in the shape None, 13, 32, and the target data by None, 7, 32. For this, we need to use the fin-tuned model to extract the data. 

We will also keep pasting the generic functions here, the ones we can use to preprocess the data easily for all the blocks below.

##### Block 0101

In [3]:
# location of the images and the correct xml files

blk_0101_data_path = '../../Spring_2024/S_lab_TasselNet/Block_3_TN/Block_3_images_and_xml'

In [4]:
content_here = os.listdir(blk_0101_data_path)
only_images = [file for file in content_here if file.split(".")[-1] == 'jpeg']

In [5]:
# len(only_images)

In [6]:
# Okay, at the end how do we need the data look like? - None, 13, 32

# What do we use for this? We use the images we have - let's only pich the horizontal images out of the lot of files we have at the location.

In [7]:
horizontal_image_list = ['2020_08_03', '2020_08_04', '2020_08_06', '2020_08_07', '2020_08_11', '2020_08_12', '2020_08_14', '2020_08_15', '2020_08_17', '2020_08_18', '2020_08_19', '2020_08_21', '2020_08_25', '2020_08_26', '2020_08_27', '2020_08_28', '2020_08_31', '2020_09_02', '2020_09_07', '2020_09_16']

In [8]:
# len(horizontal_image_list)

In [9]:
only_images[0].split('.')[0][-10:]

'2020_07_22'

In [10]:
blk_0101_horizontal_images = [file for file in only_images if file.split('.')[0][-10:] in horizontal_image_list]
# sort these files according to date
blk_0101_horizontal_images.sort()

In [11]:
# blk_0101_horizontal_images
len(blk_0101_horizontal_images)

20

In [12]:
# Write a function for extracting onlt the horizontal images out of all the files in the location

def get_horizontal_images(file_path):
    # get all contents at the file path
    all_files = os.listdir(file_path)
    # get only the image files as we do not need the xml files for the seq2seq model
    only_images = [file for file in all_files if file.split(".")[-1] == 'jpeg']
    # from all the images only choose the horizontal ones
    horizontal_images = [file for file in only_images if file.split('.')[0][-10:] in horizontal_image_list]
    # sort the list
    horizontal_images.sort()
    # return the image list
    return horizontal_images
    

In [13]:
# try out the function
blk_0101_horizontal_image_files = get_horizontal_images(blk_0101_data_path)

In [14]:
# blk_0101_horizontal_image_files
len(blk_0101_horizontal_image_files)

20

In [15]:
# function seem to work
# sanity check
np.mean(blk_0101_horizontal_images == blk_0101_horizontal_image_files)

1.0

In [16]:
# Okay, what now? We can write a function for extracting features below - we might want to think how exactly we need to stack the contents
# but notice that we do not want to store the subwindow (32,32,3) at all, and can just use the model.predict function to extract the features

In [17]:
# load the finetuned gmp model
gmp_model = tf.keras.models.load_model('../../Spring_2024/Bayes_for_comps/TS_bayes_implementation_for_TN/models/trained_gmp_model_dense_32_new.h5')

2025-06-03 17:44:28.554649: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31134 MB memory:  -> device: 0, name: Tesla V100S-PCIE-32GB, pci bus id: 0000:06:00.0, compute capability: 7.0


In [18]:
gmp_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 conv2d (Conv2D)             (None, None, None, 32)    896       
                                                                 
 activation (Activation)     (None, None, None, 32)    0         
                                                                 
 conv2d_1 (Conv2D)           (None, None, None, 32)    9248      
                                                                 
 activation_1 (Activation)   (None, None, None, 32)    0         
                                                                 
 max_pooling2d (MaxPooling2  (None, None, None, 32)    0         
 D)                                                              
                                                           

In [19]:
# define the feature extractor

# feature extractor input
feat_ext_input = gmp_model.input

# feature extractor output - do this at the ReLu activation layer - as this will give the same features as the dropout layer (It does not matter if it is the dropout or the activation layer, the extracted features will be the same)
feat_ext_output = gmp_model.layers[-4].output

feature_extractor_model = tf.keras.models.Model(inputs = feat_ext_input, outputs = feat_ext_output)

In [20]:
feature_extractor_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 conv2d (Conv2D)             (None, None, None, 32)    896       
                                                                 
 activation (Activation)     (None, None, None, 32)    0         
                                                                 
 conv2d_1 (Conv2D)           (None, None, None, 32)    9248      
                                                                 
 activation_1 (Activation)   (None, None, None, 32)    0         
                                                                 
 max_pooling2d (MaxPooling2  (None, None, None, 32)    0         
 D)                                                              
                                                             

In [21]:
# Let us read one horizontal image
joined_im_path = os.path.join(blk_0101_data_path, blk_0101_horizontal_image_files[0])
# load the image and the count numpy files
loaded_im_file = plt.imread(joined_im_path)

In [22]:
loaded_im_file.shape

(768, 1024, 3)

In [23]:
sub_window = loaded_im_file[0: 0 + 300, 0 : 0 + 300,:]
sub_window = resize(sub_window, (300, 300,3))

In [24]:
sub_window = np.expand_dims(sub_window, axis = 0)

In [25]:
sub_window.shape

(1, 300, 300, 3)

In [26]:
# try prediction from the feature extractor
try_predict = feature_extractor_model.predict(sub_window)

2025-06-03 17:44:31.005361: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907


1/1 [==============================] - 6s 6s/step


In [27]:
try_predict.shape

(1, 32)

In [28]:
try_predict

array([[0.        , 0.        , 0.10059818, 0.        , 0.07901728,
        0.        , 0.        , 0.        , 0.03111649, 0.08963878,
        0.        , 0.        , 0.10756746, 0.        , 0.14078043,
        0.        , 0.03288424, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.09947642, 0.11643474, 0.        ,
        0.03223047, 0.        , 0.06245986, 0.07843374, 0.        ,
        0.        , 0.        ]], dtype=float32)

In [29]:
sub_window_alt = loaded_im_file[0: 0 + 300, 0 : 0 + 300,:]
sub_window_alt = resize(sub_window_alt, (300, 300,3))
sub_window_alt = sub_window_alt[None,  ...]

In [30]:
sub_window_alt.shape

(1, 300, 300, 3)

In [31]:
np.mean(sub_window == sub_window_alt)

1.0

In [32]:
def create_subwindows_and_counts(im_folder_loc, image_name, conv_map, save_folder_name, stride = 300, kernel_size = 300):
    im_name = image_name + '.jpeg'
    joined_im_path = os.path.join(im_folder_loc, im_name)
    # load the image and the count numpy files
    loaded_im_file = plt.imread(joined_im_path)
        
    # create the subwindows and counts as follows
    img_height = loaded_im_file.shape[0]
    img_width = loaded_im_file.shape[1]
    
    density_sums = []
    catch_sub_image_name = []
    catch_dense_name = []
    sub_image_shapes = []
    sub_count_shapes = []
    counter = 0
    for i in  range(0, img_height, stride):
        for j in range(0, img_width, stride):
            sub_window = loaded_im_file[i: i + kernel_size, j : j + kernel_size,:]
            # resize the subwindow - for 300*300
            sub_window = resize(sub_window, (300, 300,3))
            sub_image_shapes.append(sub_window.shape)
            # save the sub window? 
            sub_window_name = image_name + '_' + str(counter) + '.npy'
            save_subwindow_path = save_folder_name + '/' + sub_window_name
            np.save(save_subwindow_path, sub_window)
            catch_sub_image_name.append(sub_window_name)